# Drug Name Entity Classifier
## AHLT - MIRI 2018



## Initialization

Load needed modules and specify the working directory

In [1]:
# Load needed packages
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,KFold # Parameter selection
import time # Execution time of some blocks
from nltk.tag import StanfordPOSTagger
import statistics
import scipy.stats # for RandomizedSearchCV


# Import our defined functions
from NER_functions import *
from datasetBuilder import *

In [2]:
# init time
first_init = time.time()

In [3]:
# Set the data directories
# train_dirs_whereto_parse = ['data/medium_train_DrugBank']
# test_dirs_whereto_parse = ['data/medium_test_DrugBank']
train_dirs_whereto_parse = ['data/Train/DrugBank','data/Train/MedLine']
test_dirs_whereto_parse = ['data/Test/DrugBankOutput','data/Test/MedlineOutput']

## Reading the train and test data from the XML files
Accessing to all the files of the directory and storing id's and text's in two arrays.
We have also added the token 'STOP' at the end of each sentence.

In [4]:
train_texts_entities = createTrainSet(train_dirs_whereto_parse)
test_texts_entities = createTestSet(test_dirs_whereto_parse)

Before computing features, I want the input data to have a special format:

In [5]:
# we want each dataset with the following format: 
# for ['have','Ibuprofeno'] ---- [('hola','V','O'),('Ibuprofeno','NN','B')]

def buildSet(text_entities):
    dataset = []
    for text,drugs in text_entities:
        # tokenizing
        tokenized_sentence = nltk.word_tokenize(text)
        print ('text: ',text)
        print('drugs: ',drugs)
        # BIO tagging
        tokens_tags = BIOTagger(text, drugs)
        # POS tagging
        tokens_pos = nltk.pos_tag(tokenized_sentence)

        text_triples = []
        for idx,token in enumerate(tokenized_sentence):
            text_triples.append((token,tokens_pos[idx][1],tokens_tags[idx][1]))
        dataset.append(text_triples)
    
    return dataset

# train_set and test_set are list of list of triples; each list of triples refers to a different sentence
train_set = buildSet(train_texts_entities) 
test_set = buildSet(test_texts_entities)

print('Number of training sentences: ',len(train_set))
print('Num of test sentences: ',len(test_set))

text:  Milk, milk products, and calcium-rich foods or drugs may impair the absorption of EMCYT.
drugs:  ['calcium', 'EMCYT']
text:  The concurrent administration of allopurinol and ampicillin increases substantially the incidence of rashes in patients receiving both drugs as compared to patients receiving ampicillin alone.
drugs:  ['allopurinol', 'ampicillin', 'ampicillin']
text:  It is not known whether this potentiation of ampicillin rashes is due to allopurinol or the hyperuricemia present in these patients.
drugs:  ['ampicillin', 'allopurinol']
text:  In controlled clinical trials of AUGMENTIN XR, 22 patients received concomitant allopurinol and AUGMENTIN XR.
drugs:  ['AUGMENTIN XR', 'allopurinol', 'AUGMENTIN XR']
text:  However, this sample size is too small to allow for any conclusions to be drawn regarding the risk of rashes with concomitant AUGMENTIN XR and allopurinol use.
drugs:  ['AUGMENTIN XR', 'allopurinol']
text:  In common with other broad-spectrum antibiotics, AUGMENTIN

text:  adjust salicylate dosage accordingly if effect is altered.
drugs:  ['salicylate']
text:  Barbiturates, phenytoin, or rifampin increased metabolic clearance of fludrocortisone acetate because of the induction of hepatic enzymes.
drugs:  ['Barbiturates', 'phenytoin', 'rifampin', 'fludrocortisone acetate']
text:  Observe the patient for possible diminished effect of steroid and increase the steroid dosage accordingly.
drugs:  ['steroid', 'steroid']
text:  Anabolic steroids (particularly C-17 alkylated androgens such as oxymetholone, methandrostenolone, norethandrolone, and similar compounds) enhanced tendency toward edema.
drugs:  ['Anabolic steroids', 'androgens', 'oxymetholone', 'methandrostenolone', 'norethandrolone']
text:  Vaccines neurological complications and lack of antibody response .
drugs:  ['Vaccines']
text:  Estrogen increased levels of corticosteroid-binding globulin, thereby increasing the bound (inactive) fraction;
drugs:  ['Estrogen']
text:  this effect is at leas

text:  Platelet inhibitors: Drugs such as acetylsalicylic acid, dextran, phenylbutazone, ibuprofen, indomethacin, dipyridamole, hydroxychloroquine and others that interfere with platelet-aggregation reactions (the main hemostatic defense of heparinized patients) may induce bleeding and should be used with caution in patients receiving heparin sodium.
drugs:  ['Platelet inhibitors', 'acetylsalicylic acid', 'dextran', 'phenylbutazone', 'ibuprofen', 'indomethacin', 'dipyridamole', 'hydroxychloroquine', 'heparin sodium']
text:  The anticoagulant effect of heparin is enhanced by concurrent treatment with antithrombin III (human) in patients with hereditary antithrombin III deficiency.
drugs:  ['heparin', 'antithrombin III']
text:  Thus in order to avoid bleeding, reduced dosage of heparin is recommended during treatment with antithrombin III (human).
drugs:  ['heparin', 'antithrombin III']
text:  Drugs Decreasing Heparin Effect: Digitalis, tetracyclines, nicotine, or antihistamines may part

text:  - Dantrolene (e.g., Dantrium) or
drugs:  ['Dantrolene', 'Dantrium']
text:  - Daunorubicin (e.g., Cerubidine) or
drugs:  ['Daunorubicin', 'Cerubidine']
text:  - Disulfiram (e.g., Antabuse) or
drugs:  ['Disulfiram', 'Antabuse']
text:  - Divalproex (e.g., Depakote) or
drugs:  ['Divalproex', 'Depakote']
text:  - Estrogens (female hormones) or
drugs:  ['Estrogens']
text:  - Etretinate (e.g., Tegison) or
drugs:  ['Etretinate', 'Tegison']
text:  - Gold salts (medicine for arthritis) or
drugs:  ['Gold']
text:  - Hydroxychloroquine (e.g., Plaquenil) or
drugs:  ['Hydroxychloroquine', 'Plaquenil']
text:  - Mercaptopurine (e.g., Purinethol) or
drugs:  ['Mercaptopurine', 'Purinethol']
text:  - Naltrexone (e.g., Trexan) (with long-term, high-dose use) or
drugs:  ['Naltrexone', 'Trexan']
text:  - Oral contraceptives (birth control pills) containing estrogen or
drugs:  ['contraceptives', 'estrogen']
text:  - Other anti-infectives by mouth or by injection (medicine for infection) or
drugs:  ['an

text:  Sodium cephalothin may enhance the nephrotoxicity of Coly-Mycin M Parenteral.
drugs:  ['Sodium cephalothin', 'Coly-Mycin M']
text:  The concomitant use of sodium cephalothin and Coly-Mycin M Parenteral should be avoided.
drugs:  ['sodium cephalothin', 'Coly-Mycin M']
text:  Hypotension - Patients on Diuretic Therapy: Patients on diuretics, and especially those in whom diuretic therapy was recently instituted, may occasionally experience an excessive reduction of blood pressure after initiation of therapy with PRINIVIL.
drugs:  ['diuretics', 'PRINIVIL']
text:  The possibility of hypotensive effects with PRINIVIL can be minimized by either discontinuing the diuretic or increasing the salt intake prior to initiation of treatment with PRINIVIL.
drugs:  ['PRINIVIL', 'diuretic', 'PRINIVIL']
text:  If it is necessary to continue the diuretic, initiate therapy with PRINIVIL at a dose of 5 mg daily, and provide close medical supervision after the initial dose until blood pressure has sta

...

text:  In study 1 (fasted), 14 volunteers (26%) experienced a total of 29 AEs; 
drugs:  []
text:  the most common nonlaboratory AEs were dizziness (6%) and headache (4%). 
drugs:  []
text:  In study 2, 12 volunteers (22%) experienced a total of 19 AEs; 
drugs:  []
text:  the most common nonlaboratory AEs were headache (17%) and dry throat (4%). 
drugs:  []
text:  AEs were generally mild or moderate in intensity. 
drugs:  []
text:  In these 2 single-dose studies, these healthy volunteers administered a single oral dose of 105-mg fenofibric acid met the US Food and Drug Administration regulatory criteria for assuming bioequivalence to a single oral dose of 145-mg fenofibrate tablets with respect to the rate and extent of fenofibric acid absorption in both fed and fasted states. 
drugs:  ['fenofibrate', 'fenofibric acid', 'fenofibric acid']
text:  Fenofibric acid at the dose studied was well tolerated in this population.
drugs:  ['Fenofibric acid']
text:  Fenofibrate is a prodrug that is 

text:  Some pharmacokinetic drug interactions are based on the competitive inhibition of the rate of drug metabolism of one of the drugs, leading to an increased concentration of the drug which was not intended. 
drugs:  []
text:  Unrecognized fatalities related to colchicine in hospitalized patients.&#xd;&#xa;
drugs:  ['colchicine']
text:  A contributing role for colchicine in causing death was unlikely in 24/37, possible in 7/37, likely in 3/37, and certain in 3/37. 
drugs:  ['colchicine']
text:  Colchicine doses (based on creatinine clearance) exceeded the accepted range for 12 patients, including 10 of 17 cases of toxicity and 8 of 13 cases of death classified as possible or higher. 
drugs:  ['Colchicine']
text:  Seventeen patients received interacting medications, including 8 of 17 cases of toxicity and 8 of 13 cases of death classified as possible or higher. 
drugs:  []
text:  Colchicine toxicity was frequent in this cohort and may have contributed to about one-third of the death

text:  an average particle size of this nanoemulsion was 41.2    7.2 nm. 
drugs:  []
text:  The permeation of colchicine in the nanoemulsion across the intestinal membrane was significantly different from that of the control group (0.2 mM colchicine). 
drugs:  ['colchicine', 'colchicine']
text:  Finally, co-administration of eugenol in colchicine nanoemulsion to enhance the colchicine bioavailability was investigated by an oral administration method. 
drugs:  ['eugenol', 'colchicine', 'colchicine']
text:  After oral administration of colchicine (8 mg/kg) in the form of either the nanoemulsion or in free colchicine solution, the relative bioavailability of nanoemulsion and eugenol-nanoemulsion were enhanced by about 1.6- and 2.1-fold, respectively, compared with free colchicine solution. 
drugs:  ['colchicine', 'colchicine', 'colchicine', 'eugenol']
text:  The procedure indicated that the intestinal absorption of colchicine was enhanced significantly by eugenol in the tested nanoemulsio

text:  Data were submitted to the statistical t-test (5%), which showed that after seven days group 2 had a significant increase in the degree of tooth bleaching compared with group 1. 
drugs:  []
text:  The mean values (  SD) were 16.33 (  3.95) for group 1 and 19.29 (  4.97) for group 2. 
drugs:  []
text:  However, the results for group 1 and group 2 were similar after 14 days. 
drugs:  []
text:  Novel insights into the synergistic interaction of Bortezomib and TRAIL: tBid provides the link.&#xd;&#xa;
drugs:  ['Bortezomib']
text:  The proteasome inhibitor Bortezomib has been identified as a potent enhancer of TRAIL-induced apoptosis in several human cancers. 
drugs:  ['Bortezomib']
text:  However, the identification of the underlying molecular mechanisms of this synergistic cell death induction has been ongoing over the last years. 
drugs:  []
text:  A recent study identifies a new mechanism of action for the synergism of TRAIL and Bortezomib.
drugs:  ['Bortezomib']
text:  Nitric oxi

text:  Observe pathological sections of stomach to study the protective effect of MWCHCP. 
drugs:  []
text:  Reverse transcription-polymerase chain reaction (RT-PCR) was applied to investigate the tumour necrosis factor-alpha (TNF-alpha) expression level of the intestine. 
drugs:  []
text:  Combining with cisplatin and MWCHCP caused a tendency of increasing the tumor growth inhibition rate and significant attenution of cisplatin-induced diarrhea, visceral organ injury, gastric mucosal injury and decreased TNF-alpha mRNA level of intestine. 
drugs:  ['cisplatin', 'cisplatin']
text:  The present findings suggest that MWCHCP increases the inhibition rate of tumor growth of cisplatin and has a beneficial influence on gastrointestinal lesion induced by cisplatin.
drugs:  ['cisplatin', 'cisplatin']
text:  CPP-ACP complex as a new adjunctive agent for remineralisation: a review.&#xd;&#xa;
drugs:  ['CPP-ACP complex']
text:  In addition to regular professional oral hygiene visits and the applic

## FEATURES 

In [6]:
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()
    
def word2features(sent, i, database):
    word = sent[i][0]
    postag = sent[i][1]
    
    # orthographic features
    features = {
    'all_uppercase_letters' : allCaps(word), 
    'initial_capital_letter': initCap(word), 
    'contains_capital_letter' : hasCap(word),
    'single_capital_letter' : singleCap(word),
    'punctuation' : punctuation(word),
    'initial_digit' : initDigit(word),
    'single_digit' : singleDigit(word),
    'letter_and_num' : alphaNum(word),
    'many_numbers' : manyNum(word),
    'contains_real_numbers' : realNum(word),
    'intermediate_dash' : inDash(word),
    'has_digit' : hasDigit(word),
    'is_Dash' : isDash(word),
    'is_roman_letter' : roman(word),
    'is_end_punctuation' : endPunctuation(word),
    'caps_mix' : capsMix(word),
        
    # Morphological information: prefixes/suffixes of lengths from 2 to 5 and word shapes of tokens. 
    'word[-5:]': word[-5:],
    'word[-4:]': word[-4:],    
    'word[-3:]': word[-3:],
    'word[-2:]': word[-2:],
    
    # Domain knowledge
    'contains_drug_sufix': containsSufix(word),
    'contains_drug_prefix': containsPrefix(word),

    # POS tag   
    'postag': postag,
    'postag[:2]': postag[:2],
        
    # Is in DrugBank dataset
    'isInDB':isTokenInDB(word,database)
    }
    
    
    
    # context features
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:unigrams' : num_ngrams(word1,1),
            '-1:bigrams' : num_ngrams(word1,2),
            '-1:trigrams' : num_ngrams(word1,3)
        })
        if i!=1:
            word2 = sent[i-2][0]
            postag2 = sent[i-2][1]
            features.update({
            '-2:postag': postag2,
            '-2:postag[:2]': postag2[:2],
            '-2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2),
                    '-2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2)})
                
            
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:unigrams' : num_ngrams(word1,1),
            '+1:bigrams' : num_ngrams(word1,2),
            '+1:trigrams' : num_ngrams(word1,3)
        })
        
        if i<len(sent)-2:
            word2 = sent[i+2][0]
            postag2 = sent[i+2][1]
            features.update({
                '+2:postag': postag2,
                '+2:postag[:2]': postag2[:2],
                '+2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2),
                    '+2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2)})
            
    else:
        features['EOS'] = True

    return features


def sent2features(sent,database):
    return [word2features(sent, i,drugbank_db) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
 # remember s is a triple defined as follows: (token, POS tag, BIO tag)
X_train = [sent2features(s,drugbank_db) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s,drugbank_db) for s in test_set]
y_test = [sent2labels(s) for s in test_set]


In [8]:
print(X_train[0])


[{'all_uppercase_letters': 0, 'initial_capital_letter': 1, 'contains_capital_letter': 1, 'single_capital_letter': 0, 'punctuation': 0, 'initial_digit': 0, 'single_digit': 0, 'letter_and_num': 0, 'many_numbers': 0, 'contains_real_numbers': 0, 'intermediate_dash': 0, 'has_digit': 0, 'is_Dash': 0, 'is_roman_letter': 0, 'is_end_punctuation': 0, 'caps_mix': 1, 'word[-5:]': 'Milk', 'word[-4:]': 'Milk', 'word[-3:]': 'ilk', 'word[-2:]': 'lk', 'contains_drug_sufix': 0, 'contains_drug_prefix': 0, 'postag': 'NN', 'postag[:2]': 'NN', 'isInDB': False, 'BOS': True, '+1:postag': ',', '+1:postag[:2]': ',', '+1:unigrams': [','], '+1:bigrams': [], '+1:trigrams': [], '+2:postag': 'NN', '+2:postag[:2]': 'NN', '+2:unigrams': ['m', 'i', 'l', 'k'], '+2:bigrams': ['mi', 'il', 'lk'], '+2:trigrams': ['mil', 'ilk']}, {'all_uppercase_letters': 0, 'initial_capital_letter': 0, 'contains_capital_letter': 0, 'single_capital_letter': 0, 'punctuation': 1, 'initial_digit': 0, 'single_digit': 0, 'letter_and_num': 0, 'man

In [9]:
y_train[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O']

In [10]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

####  Predictions

In [11]:
pred_tags = crf.predict(X_test)


####  Evaluation

In [19]:
precision = []
recall = []
for idx, text_drugs in enumerate(test_texts_entities): 
    # so text_drugs will be a tuple of (sentence, list of true entities in that sentence)
    tokens = nltk.word_tokenize(text_drugs[0])
    predicted_entities = BIOTagsToEntities(tokens,pred_tags[idx])
    precision = precision + [compute_precision(predicted_entities,text_drugs[1])]
    recall = recall + [compute_recall(predicted_entities,text_drugs[1])]
    print('tokens: ',tokens)
    print('pred tags: ',pred_tags[idx])
    print('true tags: ',y_test[idx])
    print('pred entities: ',predicted_entities)
    print('true entities:',text_drugs[1])
    print('recall: ', recall[idx])
    print('precision: ',precision[idx],'\n')
avg_precision = statistics.mean(precision)
avg_recall = statistics.mean(recall)

# F1 metric
F1 = round((2*avg_precision*avg_recall) / (avg_precision + avg_recall),2)
print(F1)

[50.0]
tokens:  ['Drug', 'Interactions', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', 'This', 'medicine', 'may', 'be', 'affected', 'by', 'the', 'drug', 'cholestyramine', 'and', 'should', 'not', 'be', 'taken', 'with', 'any', 'form', 'of', 'estrogen', 'therapy', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

pred tags:  ['B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O']
true tags:  ['B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O']
pred entities:  ['Warfarin', 'Sparfloxacin', 'warfarin']
true entities: ['warfarin', 'Sparfloxacin', 'Warfarin']
recall:  100.0
precision:  100.0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
token

precision:  0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 78.0, 100.0, 0, 0, 50.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100.0, 67.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 0, 0, 0, 0

true entities: []
recall:  0
precision:  0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 78.0, 100.0, 0, 0, 50.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100.0, 67.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 1

true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: ['o-cymen-5-ol', 'zinc']
recall:  0
precision:  0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 78.0, 100.0, 0, 0, 50.0, 0, 0, 0, 0, 0, 100.0, 

tokens:  ['In', 'addition', ',', 'other', 'disorders', ',', 'including', 'diabetic', 'or', 'alcoholic', 'ketoacidosis', ',', 'acute', 'kidney', 'injury', ',', 'chronic', 'kidney', 'disease', ',', 'and', 'lactic', 'acidosis', ',', 'can', 'cause', 'high-anion-gap', 'metabolic', 'acidosis', 'associated', 'with', 'an', 'increased', 'serum', 'osmolal', 'gap', 'and', 'therefore', 'should', 'be', 'explored', 'in', 'the', 'differential', 'diagnosis', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  ['diabetic', 'lactic acidosis']
true entities:

recall:  0
precision:  0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 78.0, 100.0, 0, 0, 50.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100.0, 67.0, 0, 0, 0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 0,

pred entities:  ['ethyl acetate', 'methanol-ammonium acetate']
true entities: ['methanol-ammonium acetate', 'ethyl acetate']
recall:  100.0
precision:  100.0 

[50.0, 0, 100.0, 0, 0.0, 100.0, 100.0, 60.0, 100.0, 100.0, 100.0, 0, 53.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 0.0, 0, 100.0, 0, 100.0, 50.0, 50.0, 0, 80.0, 100.0, 100.0, 100.0, 100.0, 80.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 0, 0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 87.0, 100.0, 86.0, 0.0, 0, 0, 0, 100.0, 50.0, 0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 100.0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 0, 100.0, 0, 100.0, 100.0, 0, 100.0, 100.0, 100.0, 100.0, 0, 50.0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 67.0, 50.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 78.0, 100.0, 0, 0, 50.0, 0, 0, 0, 0, 0, 100.0, 100.

In [13]:
print('Total execution time: ',(time.time() - first_init)/60, ' minutes')

Total execution time:  1.5198241511980692  minutes


In [14]:

'''
## Log of results
date, precision, recall, F1, features, test
14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes
'''

'\n## Log of results\ndate, precision, recall, F1, features, test\n14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes\n'